In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import matplotlib.pyplot as plt
import seaborn as sns
import xgboost
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import OrdinalEncoder
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

import xgboost




from collections import Counter

In [3]:
train = pd.read_csv('./train.csv')
test = pd.read_csv('./test.csv')

In [4]:
numerical_feats = train.dtypes[train.dtypes != "object"].index 
print("Number of numerical features: ", len(numerical_feats))

categorical_feats = train.dtypes[train.dtypes == "object"].index 
print("Number of categorical features: ", len(categorical_feats))

Number of numerical features:  2
Number of categorical features:  8


In [5]:
num_strong_corr = ['pretest']

num_weak_corr = ['n_student']

catg_strong_corr = ['lunch', 'school', 'classroom', 'school_setting']

catg_weak_corr = ['gender', 'teaching_method']

In [6]:
std_id_test = test['student_id']

to_drop_num = num_weak_corr
to_drop_catg = catg_weak_corr

cols_to_drop = ['student_id'] + to_drop_num + to_drop_catg

for df in [train, test]:
    df.drop(cols_to_drop, inplace = True, axis = 1)

In [7]:
y = train.pretest
X = train.drop(['pretest'], axis=1)

In [8]:
s = (X.dtypes == 'object')
object_cols = list(s[s].index)
object_cols

['school', 'school_setting', 'school_type', 'classroom', 'lunch']

In [9]:
label_X = X.copy()
label_test = test.copy()

ordinal_encoder = OrdinalEncoder()
label_X[object_cols] = ordinal_encoder.fit_transform(X[object_cols])
label_test[object_cols] = ordinal_encoder.transform(test[object_cols])

In [10]:
X_train, X_val, y_train, y_val = train_test_split(label_X, y, test_size=0.2, random_state=3)

In [11]:
regressor = xgboost.XGBRegressor(colsample_bytree = 0.4603, learning_rate = 0.06, min_child_weight=1.8, max_depth=3, subsample=0.52, n_estimators=2000, random_state=7)
regressor.fit(X_train, y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.4603, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.06, max_delta_step=0, max_depth=3,
             min_child_weight=1.8, missing=nan, monotone_constraints='()',
             n_estimators=2000, n_jobs=8, num_parallel_tree=1, random_state=7,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=0.52,
             tree_method='exact', validate_parameters=1, verbosity=None)

In [12]:
pred = regressor.predict(X_val)
mean_absolute_error(pred, y_val)

2.5812620809900833

In [13]:
XGB_result = regressor.predict(label_test)

In [14]:
sub_xgb = pd.DataFrame()
sub_xgb['student_id'] = std_id_test
sub_xgb['pretest'] = XGB_result

sub_xgb.to_csv('xgb.csv', index=False)